# Aula 06 - Dados nD - netCDF

Nesta aula trabalhamos com dados 3 ou 4 D em formato netCDF. 
Utilizamos o pacote https://docs.xarray.dev/en/stable/ e  trabalharemos com os dados da plataforma
https://disc.gsfc.nasa.gov/

## Baixando os dados do GESDISC
- Crie um login para você
- Digite uma palavra-chave que descreva o dado que desejar
- Baixe o dado para um determinado período
- https://disc.gsfc.nasa.gov/datasets/MERRA2_CNN_HAQAST_PM25_1/summary?keywords=air%20quality
- https://github.com/nasa/gesdisc-tutorials/blob/main/notebooks/How_to_Access_GES_DISC_Data_Using_Python.ipynb
- 



In [6]:
# Caminhos para a pasta de dados
import os 

# Pasta atual
pastaAtual = os.getcwd()
print(pastaAtual)

# Pasta de dados
pastaDados = r'C:\Users\Leonardo.Hoinaski\Documents\ENS410064\2024\dados'
listaDados = os.listdir(pastaDados)

# Listando arquivos
print(listaDados)

# Selecionando arquivos do MERRA2
prefixed = [filename for filename in os.listdir(pastaDados) if filename.startswith("MERRA2")]
print(prefixed)

C:\Users\Leonardo.Hoinaski\Documents\ENS410064\2024\notebooks
['1728675725931.csv', 'BR_Pais_2022', 'BR_Pais_2022.zip', 'BR_UF_2022', 'BR_UF_2022.zip', 'Deg2UTM.csv', 'EZ_AERMOD_leohoinaski_2024-11-14', 'EZ_AERMOD_leohoinaski_2024-11-14.zip', 'FINNv2.5.1_modvrs_nrt_GEOSCHEM_20240708.txt', 'MERRA2_HAQAST_CNN_L4_V1.20240505.nc4', 'MERRA2_HAQAST_CNN_L4_V1.20240506.nc4', 'metar_FORT_2023.csv', 'metar_SBFL_2022.csv', 'metar_SBFL_2023.csv', 'metar_SBFL_2024.csv', 'SC_distritos_CD2022', 'SC_distritos_CD2022.zip', 'UPPERAIR(1).UA']
['MERRA2_HAQAST_CNN_L4_V1.20240505.nc4', 'MERRA2_HAQAST_CNN_L4_V1.20240506.nc4']


In [7]:
# Abrir arquivos do MERRA utilizando o xarray
import xarray as xr

dataset = xr.Dataset()